In [1]:
import multiprocessing as mp

In [2]:
from selenium import webdriver

In [2]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import csv

In [4]:
import requests as req

from bs4 import BeautifulSoup as bs

In [5]:
import time   # para manejar tiempo

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By   # para buscar segun tag, id, class, etc...

from selenium.webdriver.support.ui import WebDriverWait   # es para esperar

from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...

from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click

from selenium.webdriver.common.keys import Keys  # manejar teclas


import warnings
warnings.filterwarnings('ignore')

### We start loading the clean Database we already have

In [6]:
path = f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\cause_of_deaths.csv'
data_cause = pd.read_csv(path)



In [7]:
data_cause.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6120 entries, 0 to 6119
Data columns (total 34 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Country/Territory                           6120 non-null   object
 1   Code                                        6120 non-null   object
 2   Year                                        6120 non-null   int64 
 3   Meningitis                                  6120 non-null   int64 
 4   Alzheimer's Disease and Other Dementias     6120 non-null   int64 
 5   Parkinson's Disease                         6120 non-null   int64 
 6   Nutritional Deficiencies                    6120 non-null   int64 
 7   Malaria                                     6120 non-null   int64 
 8   Drowning                                    6120 non-null   int64 
 9   Interpersonal Violence                      6120 non-null   int64 
 10  Maternal Disorders      

### Now we need webscraping to extract population data from the Internet

In [10]:
from joblib import Parallel, delayed

In [11]:
from webdriver_manager.chrome import ChromeDriverManager

In [15]:
url = "http://www.geoba.se/population.php?pc=world&type=28&year=2000&st=country&asde=&page=1"

In [16]:
html=req.get(url).text
soup=bs(html, 'html.parser')

In [17]:
rows = soup.find_all("tr",class_ = 'bb')

In [18]:
len(rows)

100

In [19]:
len(rows[0]("td"))

400

In [9]:
lst_url = []
for e in range(2000,2010):
    for i in range(1,4):
        lst_url.append(f"http://www.geoba.se/population.php?pc=world&type=28&year={str(e)}&st=country&asde=&page={str(i)}")
    

In [10]:
lst_url2 = []
for e in range(2010,2020):
    for i in range(1,4):
        lst_url.append(f"http://www.geoba.se/population.php?pc=world&type=28&year={str(e)}&st=country&asde=&page={str(i)}")

In [ ]:
type(int(len(lst_url)/2))

In [ ]:
from selenium.webdriver.support.wait import WebDriverWait

### We create a scrapping function with a list of urls as input

In [11]:
%%time
def extraer(url):
    try:
        html=req.get(url).text
        soup=bs(html, 'html.parser')
        lst = []
        rows = soup.find_all("tr",class_ = 'bb')
        time.sleep(2)
        for i in range(len(rows)):
            data= rows[i]("td")
            lst1 = []
            for j in range(4):
                time.sleep(1)
                
                lst1.append(data[j].text)
            lst.append(lst1)
        
        return pd.DataFrame(lst, columns=["rank","country","population","year"])
    except:
        print(url)

    
    
    

CPU times: total: 0 ns
Wall time: 0 ns


In [12]:
mp.cpu_count()

8

In [ ]:
html=req.get(url).text
soup=bs(html, 'html.parser')
lst = []
rows = soup.find_all("tr",class_ = 'bb')

In [13]:
from tqdm.notebook import tqdm

### Parallelization of scrapping accelerates the process

In [14]:
lst_df=Parallel(n_jobs=7, verbose=True)(delayed(extraer)(url) for url in tqdm(lst_url))

  0%|          | 0/60 [00:00<?, ?it/s]

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed: 29.1min
[Parallel(n_jobs=7)]: Done  60 out of  60 | elapsed: 46.6min finished


In [29]:
len(lst_df)

60

In [ ]:
lst_df[0]

In [36]:
lst_df[32]

,rank,country,population,year
0,20.,Thailand,"66,336,256",2010
1,177.,The Bahamas,"310,426",2010
2,148.,The Gambia,"1,755,456",2010
3,100.,Togo,"6,587,239",2010
4,192.,Tonga,"105,632",2010
5,156.,Trinidad and Tobago,"1,228,691",2010
6,78.,Tunisia,"10,525,041",2010
7,17.,Turkey,"77,804,120",2010
8,116.,Turkmenistan,"4,940,916",2010
9,211.,Turks and Caicos Islands,"43,212",2010


In [39]:
f = [lst_df[0],lst_df[1],lst_df[2]]

In [99]:
g = [lst_df[57],lst_df[58],lst_df[59]]

In [45]:
lsta2 = pd.concat(g, ignore_index= True) 

In [52]:
lsta3 = pd.concat(g, ignore_index= True) 

In [55]:
lsta4 = pd.concat(g, ignore_index= True) 

In [58]:
lsta5 = pd.concat(g, ignore_index= True) 

In [60]:
lsta6 = pd.concat(g, ignore_index= True) 

In [64]:
lsta7 = pd.concat(g, ignore_index= True) 

In [67]:
lsta8 = pd.concat(g, ignore_index= True) 

In [69]:
lsta9 = pd.concat(g, ignore_index= True) 

In [72]:
lsta10 = pd.concat(g, ignore_index= True) 

In [74]:
lsta11 = pd.concat(g, ignore_index= True) 

In [78]:
lsta12 = pd.concat(g, ignore_index= True) 

In [80]:
lsta13 = pd.concat(g, ignore_index= True) 

In [82]:
lsta14 = pd.concat(g, ignore_index= True) 

In [87]:
lsta15 = pd.concat(g, ignore_index= True) 

In [89]:
lsta16 = pd.concat(g, ignore_index= True) 

In [93]:
lsta17 = pd.concat(g, ignore_index= True) 

In [96]:
lsta18 = pd.concat(g, ignore_index= True) 

In [98]:
lsta19 = pd.concat(g, ignore_index= True) 

In [100]:
lsta20 = pd.concat(g, ignore_index= True) 

In [249]:
def integr(x):
    
        
    for i in range(len(x)):
        try:
    
        
        
            
            x[i].rank = x[i].rank.replace(".","")
            x[i]= x[i].astype({'population':'int64','year':'int32','rank':'int32'})
        
        except:
            return print(x[i])
       
        
    
        
            
        

In [202]:
def integ(x):
    try:
        
        for j in range(len(x)):
        
        
            x.population[j] = int(x.population[j])
            x.year[j] = int(x.year[j])
            
    except:
        print("error")
        
        

In [190]:
for j in range(len(lista_all[18])):
    lista_all[17].population[j]= lista_all[17].population[j].replace(",","")

In [184]:
lista_all= [lsta1,lsta2,lsta3,lsta4,lsta5,lsta6,lsta7,lsta8,lsta9,lsta10,lsta11,lsta12,lsta13,lsta14,lsta15,lsta16,lsta17,lsta18,lsta19,lsta20]

In [ ]:
lista_all[i].population[j] = int(lista_all[i].population[j].replace(",",""))
        lista_all[i].year[j] = int(lista_all[i].year[j])

In [252]:
for i in range(len(lista_all)):
    lista_all[i].to_csv(f'C:\\Users\\pabli\\apuntes_de_clase\\Proyecto-ETL\data\\lsta{i}.csv')

In [3]:
all_ =pd.read_csv(f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\lsta_all.csv')

In [4]:
all_.drop(columns = ["Unnamed: 0"], inplace=True)

In [5]:
all_sort=all_.sort_values(by=["country","year"])

In [6]:
all_sort.reset_index(inplace=True)

In [7]:
all_sort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       4560 non-null   int64  
 1   rank        4560 non-null   float64
 2   country     4560 non-null   object 
 3   population  4560 non-null   int64  
 4   year        4560 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 178.2+ KB


In [8]:
all_sort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       4560 non-null   int64  
 1   rank        4560 non-null   float64
 2   country     4560 non-null   object 
 3   population  4560 non-null   int64  
 4   year        4560 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 178.2+ KB


### We sort the concatenated data from our scrapping

In [9]:
for i in range(19,len(all_sort.year),20):
    all_sort.year[i]= 2019
    

C:\Users\pabli\AppData\Local\Temp\ipykernel_1468\1331422615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sort.year[i]= 2019


In [11]:
all_sort.to_csv(f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\all_sort.csv')

In [30]:
all_sort.country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad',
       'Channel Islands', 'Chile', 'China', 'Colombia', 'Comoros',
       'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Democratic Republic of the Congo',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic',
       'East Timor', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia',
       'Faroe Islands', 'Fe

In [31]:
data_cause.rename(columns = {"Country/Territory":"country","Year":"year"},inplace=True)

### New dataframe is created merging the base data with population data for the matching registers

In [138]:
data_int = pd.merge(data_cause, all_sort, how="outer", on = ["country","year"])

### The new merge is removed of any nulls and registers with inconsistent data types

In [139]:
nan_cols=data_int.isna().mean() * 100
nan_cols

country                                        0.000000
Code                                          11.304348
year                                           0.000000
Meningitis                                    11.304348
Alzheimer's Disease and Other Dementias       11.304348
Parkinson's Disease                           11.304348
Nutritional Deficiencies                      11.304348
Malaria                                       11.304348
Drowning                                      11.304348
Interpersonal Violence                        11.304348
Maternal Disorders                            11.304348
HIV/AIDS                                      11.304348
Drug Use Disorders                            11.304348
Tuberculosis                                  11.304348
Cardiovascular Diseases                       11.304348
Lower Respiratory Infections                  11.304348
Neonatal Disorders                            11.304348
Alcohol Use Disorders                         11

In [140]:
data_int['num_nan']=[sum([pd.isna(e) for e in fila]) for fila in data_int.itertuples()]

In [141]:
data_int.shape

(6900, 38)

In [368]:
data_int.head()

,index,country,Code,year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis,rank,population,per_capita-Meningitis,per_capita-Alzheimer's Disease and Other Dementias,per_capita-Parkinson's Disease,per_capita-Nutritional Deficiencies,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-Maternal Disorders,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Tuberculosis,per_capita-Cardiovascular Diseases,per_capita-Lower Respiratory Infections,per_capita-Neonatal Disorders,per_capita-Alcohol Use Disorders,per_capita-Self-harm,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Neoplasms,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Chronic Kidney Disease,per_capita-Poisonings,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Digestive Diseases,"per_capita-Fire, Heat, and Hot Substances",per_capita-Acute Hepatitis
0,10,Afghanistan,AFG,2000,3105.0,1291.0,416.0,3060.0,263.0,2048.0,2871.0,4271.0,97.0,186.0,5864.0,53919.0,32395.0,22920.0,99.0,1137.0,0.0,11323.0,52.0,13977.0,5429.0,2515.0,4182.0,499.0,3013.0,6581.0,7471.0,3249.0,6206.0,465.0,4334.0,46.0,22461348.0,0.000138,0.000057,0.000019,0.000136,0.000012,0.000091,0.000128,0.000190,0.000004,0.000008,0.000261,0.002401,0.001442,0.001020,0.000004,0.000051,0.000000e+00,0.000504,0.000002,0.000622,0.000242,0.000112,0.000186,0.000022,0.000134,0.000293,0.000333,0.000145,0.000276,0.000021,0.000193
1,11,Afghanistan,AFG,2001,3063.0,1300.0,424.0,2973.0,211.0,2044.0,2971.0,4385.0,104.0,197.0,5900.0,54642.0,31594.0,23276.0,101.0,1183.0,4.0,11446.0,380.0,14312.0,5689.0,2657.0,4251.0,502.0,2924.0,6720.0,7565.0,3267.0,6277.0,467.0,4363.0,46.0,22507460.0,0.000136,0.000058,0.000019,0.000132,0.000009,0.000091,0.000132,0.000195,0.000005,0.000009,0.000262,0.002428,0.001404,0.001034,0.000004,0.000053,1.777189e-07,0.000509,0.000017,0.000636,0.000253,0.000118,0.000189,0.000022,0.000130,0.000299,0.000336,0.000145,0.000279,0.000021,0.000194
2,12,Afghanistan,AFG,2002,2990.0,1311.0,429.0,2795.0,2883.0,1980.0,3277.0,4498.0,111.0,202.0,5660.0,54213.0,30346.0,23976.0,101.0,1190.0,1286.0,11393.0,53.0,14437.0,977.0,2745.0,4239.0,490.0,2746.0,6696.0,7455.0,3238.0,6246.0,455.0,4251.0,45.0,23600088.0,0.000127,0.000056,0.000018,0.000118,0.000122,0.000084,0.000139,0.000191,0.000005,0.000009,0.000240,0.002297,0.001286,0.001016,0.000004,0.000050,5.449132e-05,0.000483,0.000002,0.000612,0.000041,0.000116,0.000180,0.000021,0.000116,0.000284,0.000316,0.000137,0.000265,0.000019,0.000180
3,13,Afghanistan,AFG,2003,3300.0,1326.0,434.0,3039.0,2188.0,2282.0,3458.0,4702.0,118.0,216.0,5676.0,54274.0,32361.0,24628.0,104.0,1235.0,137.0,12348.0,55.0,14951.0,864.0,2865.0,4336.0,531.0,2984.0,7243.0,7471.0,3342.0,6443.0,499.0,4373.0,42.0,25005388.0,0.000132,0.000053,0.000017,0.000122,0.000088,0.000091,0.000138,0.000188,0.000005,0.000009,0.000227,0.002170,0.001294,0.000985,0.000004,0.000049,5.478819e-06,0.000494,0.000002,0.000598,0.000035,0.000115,0.000173,0.000021,0.000119,0.000290,0.000299,0.000134,0.000258,0.000020,0.000175
4,14,Afghanistan,AFG,2004,3350.0,1344.0,439.0,3033.0,773.0,2386.0,3479.0,4799.0,125.0,229.0,5607.0,54438.0,32222.0,24780.0,107.0,1283.0,18.0,12258.0,57.0,15360.0,1049.0,2

In [143]:
data_int.drop(data_int[data_int.num_nan>=3].index, axis=0, inplace=True)

In [144]:
data_int.shape

(3780, 38)

In [145]:
data_int.drop(columns=["num_nan"],inplace=True)

In [146]:
data_int.drop(columns=["index"],inplace=True)

In [147]:
data_int.reset_index(inplace=True)

### Per capita values are created 

In [149]:
for e in data_int:
    for g in range(len(data_int)):
         if isinstance(data_int[e][g], float) and e != "population":
                data_int[f"per_capita-{e}"]= (data_int[e])/data_int["population"]
                
        
    
    
       
    

In [157]:
data_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3780 entries, 0 to 3779
Data columns (total 68 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   index                                                  3780 non-null   int64  
 1   country                                                3780 non-null   object 
 2   Code                                                   3780 non-null   object 
 3   year                                                   3780 non-null   int64  
 4   Meningitis                                             3780 non-null   float64
 5   Alzheimer's Disease and Other Dementias                3780 non-null   float64
 6   Parkinson's Disease                                    3780 non-null   float64
 7   Nutritional Deficiencies                               3780 non-null   float64
 8   Malaria                                         

In [152]:
data_int.drop(columns=["per_capita-rank"],inplace=True)

In [153]:
pd.set_option('display.max_columns', None)

In [367]:
data_int.head()

,index,country,Code,year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis,rank,population,per_capita-Meningitis,per_capita-Alzheimer's Disease and Other Dementias,per_capita-Parkinson's Disease,per_capita-Nutritional Deficiencies,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-Maternal Disorders,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Tuberculosis,per_capita-Cardiovascular Diseases,per_capita-Lower Respiratory Infections,per_capita-Neonatal Disorders,per_capita-Alcohol Use Disorders,per_capita-Self-harm,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Neoplasms,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Chronic Kidney Disease,per_capita-Poisonings,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Digestive Diseases,"per_capita-Fire, Heat, and Hot Substances",per_capita-Acute Hepatitis
0,10,Afghanistan,AFG,2000,3105.0,1291.0,416.0,3060.0,263.0,2048.0,2871.0,4271.0,97.0,186.0,5864.0,53919.0,32395.0,22920.0,99.0,1137.0,0.0,11323.0,52.0,13977.0,5429.0,2515.0,4182.0,499.0,3013.0,6581.0,7471.0,3249.0,6206.0,465.0,4334.0,46.0,22461348.0,0.000138,0.000057,0.000019,0.000136,0.000012,0.000091,0.000128,0.000190,0.000004,0.000008,0.000261,0.002401,0.001442,0.001020,0.000004,0.000051,0.000000e+00,0.000504,0.000002,0.000622,0.000242,0.000112,0.000186,0.000022,0.000134,0.000293,0.000333,0.000145,0.000276,0.000021,0.000193
1,11,Afghanistan,AFG,2001,3063.0,1300.0,424.0,2973.0,211.0,2044.0,2971.0,4385.0,104.0,197.0,5900.0,54642.0,31594.0,23276.0,101.0,1183.0,4.0,11446.0,380.0,14312.0,5689.0,2657.0,4251.0,502.0,2924.0,6720.0,7565.0,3267.0,6277.0,467.0,4363.0,46.0,22507460.0,0.000136,0.000058,0.000019,0.000132,0.000009,0.000091,0.000132,0.000195,0.000005,0.000009,0.000262,0.002428,0.001404,0.001034,0.000004,0.000053,1.777189e-07,0.000509,0.000017,0.000636,0.000253,0.000118,0.000189,0.000022,0.000130,0.000299,0.000336,0.000145,0.000279,0.000021,0.000194
2,12,Afghanistan,AFG,2002,2990.0,1311.0,429.0,2795.0,2883.0,1980.0,3277.0,4498.0,111.0,202.0,5660.0,54213.0,30346.0,23976.0,101.0,1190.0,1286.0,11393.0,53.0,14437.0,977.0,2745.0,4239.0,490.0,2746.0,6696.0,7455.0,3238.0,6246.0,455.0,4251.0,45.0,23600088.0,0.000127,0.000056,0.000018,0.000118,0.000122,0.000084,0.000139,0.000191,0.000005,0.000009,0.000240,0.002297,0.001286,0.001016,0.000004,0.000050,5.449132e-05,0.000483,0.000002,0.000612,0.000041,0.000116,0.000180,0.000021,0.000116,0.000284,0.000316,0.000137,0.000265,0.000019,0.000180
3,13,Afghanistan,AFG,2003,3300.0,1326.0,434.0,3039.0,2188.0,2282.0,3458.0,4702.0,118.0,216.0,5676.0,54274.0,32361.0,24628.0,104.0,1235.0,137.0,12348.0,55.0,14951.0,864.0,2865.0,4336.0,531.0,2984.0,7243.0,7471.0,3342.0,6443.0,499.0,4373.0,42.0,25005388.0,0.000132,0.000053,0.000017,0.000122,0.000088,0.000091,0.000138,0.000188,0.000005,0.000009,0.000227,0.002170,0.001294,0.000985,0.000004,0.000049,5.478819e-06,0.000494,0.000002,0.000598,0.000035,0.000115,0.000173,0.000021,0.000119,0.000290,0.000299,0.000134,0.000258,0.000020,0.000175
4,14,Afghanistan,AFG,2004,3350.0,1344.0,439.0,3033.0,773.0,2386.0,3479.0,4799.0,125.0,229.0,5607.0,54438.0,32222.0,24780.0,107.0,1283.0,18.0,12258.0,57.0,15360.0,1049.0,2

In [135]:
len(data_int)

3780

In [158]:
data_int.to_csv(f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\data_int.csv')

### This dataframe was extracted with the help of an instant data scraper

In [160]:
path =f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\2000-2009.csv'
data = pd.read_csv(path)

In [202]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  196 non-null    int64 
 1   name        196 non-null    object
 2   2000        194 non-null    object
 3   2001        194 non-null    object
 4   2002        194 non-null    object
 5   2003        194 non-null    object
 6   2004        194 non-null    object
 7   2005        194 non-null    object
 8   2006        194 non-null    object
 9   2007        194 non-null    object
 10  2008        194 non-null    object
 11  2009        194 non-null    object
dtypes: int64(1), object(11)
memory usage: 18.5+ KB


In [206]:
data.head(50)

,name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Afghanistan,"4,367","4,553","5,146","6,167","6,925","8,556","10,297","12,066","12,066","12,066"
1,Albania,"3,483","3,928","4,348","5,611","7,185","8,052","8,896","10,677","12,881","12,044"
2,Algeria,"54,749","54,745","56,761","67,864","85,332","103,198","117,027","134,977","171,001","137,211"
3,Andorra,"1,429","1,547","1,758","2,362","2,896","3,158","3,456","3,952","4,082","3,675"
4,Angola,"11,166","10,930","15,286","17,813","23,552","36,971","52,381","65,266","88,539","70,307"
5,Antigua and Barbuda,826,800,814,856,920,"1,023","1,158","1,313","1,370","1,228"
6,Argentina,"317,759","300,421","112,458","142,431","164,911","199,273","232,892","287,921","363,545","334,633"
7,Armenia,"1,912","2,118","2,376","2,807","3,577","4,900","6,384","9,206","11,662","8,648"
8,Aruba,"1,873","1,896","1,962","2,044","2,255","2,360","2,470","2,678","2,843","2,554"
9,Australia,"399,650","377,526","425,115","540,950","658,379","735,633","782,398","949,037","1,055,936","999,994"


In [196]:
bad_index_year=data["name"][data["name"].isna()].index
bad_index_year[:10]

Int64Index([39], dtype='int64')

In [215]:
data.iloc[[0, 102, 121,157,159]]

,name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Afghanistan,4367,4553,5146,6167,6925,8556,10297,12066,12066,12066
102,Macau,6860,7372,8247,10643,12160,14874,18440,21027,21588,21588
121,Nauru,32,31,29,23,37,44,44,44,44,44
157,Somalia,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000
159,South Sudan,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000


In [214]:
data.iloc[[39]]

,name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
39,Democratic Republic of Congo,19077,7246,8720,9022,10275,12069,14376,16778,19548,18495


In [171]:
data.name[102]

'Macau'

### Fill missing values with outside information or previous registers

In [ ]:
data.fillna()

In [200]:
data["name"][39]= "Democratic Republic of Congo"

In [179]:
data["2009"][102]= data["2008"][102]

In [190]:
data["2009"][121]= data["2005"][121]

In [204]:
data.drop(columns=["Unnamed: 0"],inplace=True)

In [213]:
for i in data:
    if i != "name":
        data[i][157]= "4000"
        data[i][159]= "6000"
        

In [219]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    196 non-null    object
 1   2000    196 non-null    int64 
 2   2001    196 non-null    int64 
 3   2002    196 non-null    int64 
 4   2003    196 non-null    int64 
 5   2004    196 non-null    int64 
 6   2005    196 non-null    int64 
 7   2006    196 non-null    int64 
 8   2007    196 non-null    int64 
 9   2008    196 non-null    int64 
 10  2009    196 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 17.0+ KB


In [210]:
for i,e in enumerate(data):
    for g in range(len(data[e])):
     
        data[e][g] = data[e][g].replace(",","")

In [218]:
for i in data:
    if i != "name":
         data = data.astype({i:"int64"})

In [234]:
data.head()

,name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Afghanistan,4367,4553,5146,6167,6925,8556,10297,12066,12066,12066
1,Albania,3483,3928,4348,5611,7185,8052,8896,10677,12881,12044
2,Algeria,54749,54745,56761,67864,85332,103198,117027,134977,171001,137211
3,Andorra,1429,1547,1758,2362,2896,3158,3456,3952,4082,3675
4,Angola,11166,10930,15286,17813,23552,36971,52381,65266,88539,70307


### Columns representing years are switched to being rows with the corresponding country

In [247]:
lst = []
for i,e in enumerate(data.name):
    for g in range(1,11):
        amount= data.iloc[i,g]
        year = g +1999
        lst.append({"name":e,"year":year,"gdp":amount})
data2 = pd.DataFrame(lst)
    
    
    

In [249]:
data2.to_csv(f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\econ_data1.csv')

In [251]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1960 entries, 0 to 1959
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    1960 non-null   object
 1   year    1960 non-null   int64 
 2   gdp     1960 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 46.1+ KB


### Same process as in the previous extraction

In [254]:
path2 = r'C:\Users\pabli\apuntes_de_clase\Proyecto-ETL\data\2010-2019.csv'
data3 = pd.read_csv(path2)

In [255]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   thumbborder src                 195 non-null    object
 1   tablescraper-selected-row       195 non-null    object
 2   tablescraper-selected-row href  195 non-null    object
 3   tablescraper-selected-row 2     196 non-null    object
 4   tablescraper-selected-row 3     195 non-null    object
 5   tablescraper-selected-row 4     195 non-null    object
 6   tablescraper-selected-row 5     195 non-null    object
 7   tablescraper-selected-row 6     195 non-null    object
 8   tablescraper-selected-row 7     195 non-null    object
 9   tablescraper-selected-row 8     195 non-null    object
 10  tablescraper-selected-row 9     195 non-null    object
 11  tablescraper-selected-row 10    195 non-null    object
 12  tablescraper-selected-row 11    193 non-null    ob

In [259]:
data3.drop(columns = ["thumbborder src","tablescraper-selected-row href"],inplace=True)

In [261]:
data3.columns = ["name","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"]

In [283]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    196 non-null    object
 1   2010    196 non-null    object
 2   2011    196 non-null    object
 3   2012    196 non-null    object
 4   2013    196 non-null    object
 5   2014    196 non-null    object
 6   2015    196 non-null    object
 7   2016    196 non-null    object
 8   2017    196 non-null    object
 9   2018    196 non-null    object
 10  2019    194 non-null    object
dtypes: object(11)
memory usage: 17.0+ KB


In [265]:
bad_index_year=data3["name"][data3["name"].isna()].index
bad_index_year[:10]

Int64Index([39], dtype='int64')

In [266]:
data3["name"][39]= "Democratic Republic of Congo"

In [270]:
bad_index_year=data3["2018"][data3["2018"].isna()].index
bad_index_year[:10]

Int64Index([169], dtype='int64')

In [282]:
data3.iloc[169]

name    Syria
2010    40000
2011    40000
2012    40000
2013    40000
2014    40000
2015    40000
2016    40000
2017    40000
2018    40000
2019    40000
Name: 169, dtype: object

In [281]:
for i in data3:
    if i != "name":
        data3[i][169]= "40000"

In [280]:
data3['2010']


0       15,325
1       11,937
2      161,207
3        3,446
4       83,799
        ...   
191    143,212
192      9,682
193     30,907
194     20,264
195     12,042
Name: 2010, Length: 196, dtype: object

In [284]:
bad_index_year=data3["2019"][data3["2019"].isna()].index
bad_index_year[:10]

Int64Index([157, 159], dtype='int64')

In [285]:
data3.iloc[[159,157]]

,name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
159,South Sudan,"17,186","11,267","14,947","15,383","14,801","3,501","3,495","4,659","5,319",NaN
157,Somalia,"3,917","4,042","4,577","5,025","5,335","5,534","5,609","5,856","6,485",NaN


In [287]:
data3["2019"][157]=data3["2018"][157]
data3["2019"][159]=data3["2018"][159]

In [292]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    196 non-null    object
 1   2010    196 non-null    int64 
 2   2011    196 non-null    int64 
 3   2012    196 non-null    int64 
 4   2013    196 non-null    int64 
 5   2014    196 non-null    int64 
 6   2015    196 non-null    int64 
 7   2016    196 non-null    int64 
 8   2017    196 non-null    int64 
 9   2018    196 non-null    int64 
 10  2019    196 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 17.0+ KB


In [289]:
for i,e in enumerate(data3):
    for g in range(len(data3[e])):
     
        data3[e][g] = data3[e][g].replace(",","")

In [291]:
for i in data3:
    if i != "name":
         data3 = data3.astype({i:"int64"})

In [293]:
lst = []
for i,e in enumerate(data3.name):
    for g in range(1,11):
        amount= data3.iloc[i,g]
        year = g +2009
        lst.append({"name":e,"year":year,"gdp":amount})
data4 = pd.DataFrame(lst)
    

In [301]:
data4.rename(columns = {"name":"country"},inplace=True)

In [303]:
data4.to_csv(f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\econ_data2.csv')

In [319]:
g = [data2,data4]

In [317]:
data2.rename(columns = {"name":"country"},inplace=True)

In [320]:
combined_econ = pd.concat(g, ignore_index= True) 

### Last merge of the full database with the concatenated economic data

In [322]:
data_full = pd.merge(data_int, combined_econ, how="outer", on = ["country","year"])

In [332]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3780 entries, 0 to 3779
Data columns (total 70 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   index                                                  3780 non-null   float64
 1   country                                                3780 non-null   object 
 2   Code                                                   3780 non-null   object 
 3   year                                                   3780 non-null   int64  
 4   Meningitis                                             3780 non-null   float64
 5   Alzheimer's Disease and Other Dementias                3780 non-null   float64
 6   Parkinson's Disease                                    3780 non-null   float64
 7   Nutritional Deficiencies                               3780 non-null   float64
 8   Malaria                                         

In [327]:
data_full['num_nan']=[sum([pd.isna(e) for e in fila]) for fila in data_full.itertuples()]

In [331]:
data_full.drop(data_full[data_full.num_nan>=4].index, axis=0, inplace=True)

In [337]:
data_full.describe()

,index,year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis,rank,population,per_capita-Meningitis,per_capita-Alzheimer's Disease and Other Dementias,per_capita-Parkinson's Disease,per_capita-Nutritional Deficiencies,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-Maternal Disorders,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Tuberculosis,per_capita-Cardiovascular Diseases,per_capita-Lower Respiratory Infections,per_capita-Neonatal Disorders,per_capita-Alcohol Use Disorders,per_capita-Self-harm,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Neoplasms,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Chronic Kidney Disease,per_capita-Poisonings,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Digestive Diseases,"per_capita-Fire, Heat, and Hot Substances",per_capita-Acute Hepatitis,gdp
count,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3.780000e+03,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3.780000e+03,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3.780000e+03,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3.780000e+03,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3.780000e+03,3780.000000,3.780000e+03,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000,3.780000e+03,3.560000e+03
mean,3034.626984,2009.500000,1614.887302,5441.610582,1311.118254,1733.437566,3909.002910,1541.327778,2138.108201,1198.150000,6981.840741,313.591534,7151.746825,7.874302e+04,13044.281217,12273.859524,822.165608,3895.216667,279.689947,9911.330688,293.250529,4.020717e+04,467.293651,5826.365873,5358.358730,437.208730,1495.464815,6150.917196,1.786722e+04,6642.883862,11425.523810,587.900529,572.933333,101.947090,3.392431e+07,0.000058,0.000181,0.000044,0.000069,0.000188,0.000042,0.000080,0.000041,0.000384,0.000009,0.000187,0.002472,0.000459,0.000344,0.000029,0.000113,1.100991e-05,0.000255,8.729751e-06,0.001293,0.000021,0.000273,0.000194,0.000011,0.000065,0.000175,0.000321,0.000197,0.000348,0.000021,9.735666e-06,2.614945e+05
std,1757.405599,5.767044,6176.735297,19921.786705,5105.765133,6255.780491,18531.751261,7517.296360,7045.903797,5324.470492,24281.117663,1439.009713,37262.709070,3.247856e+05,43454.105286,53854.308862,3666.396596,18179.765515,5470.916916,61067.291036,1678.488620,1.766935e+05,3233.200950,19131.718748,18260.736686,2206.634008,4975.870132,26353.118959,1.085575e+05,22489.951953,39272.121524,2167.034977,3981.023914,61.214128,1.309866e+08,0.000107,0.000164,0.000034,0.000157,0.000482,0.000034,0.000105,0.000061,0.001081,0.000015,0.000298,0.001782,0.000371,0.000411,0.000048,0.000091,3.801925e-04,0.000459,1.982380e-05,0.000953,0.000147,0.000270,0.000111,0.000013,0.000154,0.000112,0.000194,0.000110,0.000148,0.000034,1.690091e-05,8.342600e+05
min,10.000000,2000.000000,0.000000,0.0

In [336]:
data_full.drop(columns=["num_nan"],inplace=True)

In [ ]:
data_full.drop(columns=["index"],inplace=True)

In [343]:
for i,e in enumerate(data_full.gdp):
    data_full.gdp[i] = e * 1000000

In [345]:
lst = []
for i,e in enumerate(data_full.gdp):
    per_cap= e / data_full.population[i]
    lst.append(per_cap)
     
    

In [346]:
data_full["gdp/percap"]= lst

In [352]:
data_full.corr().tail(30)

,year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis,rank,population,per_capita-Meningitis,per_capita-Alzheimer's Disease and Other Dementias,per_capita-Parkinson's Disease,per_capita-Nutritional Deficiencies,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-Maternal Disorders,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Tuberculosis,per_capita-Cardiovascular Diseases,per_capita-Lower Respiratory Infections,per_capita-Neonatal Disorders,per_capita-Alcohol Use Disorders,per_capita-Self-harm,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Neoplasms,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Chronic Kidney Disease,per_capita-Poisonings,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Digestive Diseases,"per_capita-Fire, Heat, and Hot Substances",per_capita-Acute Hepatitis,gdp,gdp/percap
per_capita-Nutritional Deficiencies,-0.133445,0.128489,-0.072961,-0.063190,0.390987,0.137515,-0.016272,-0.010093,0.083513,0.132647,-0.062998,0.049167,-0.053759,0.039360,0.055792,-0.056987,-0.036304,-0.000403,0.050344,-0.033187,-0.054668,0.030985,-0.032294,-0.040221,-0.003869,0.486928,-0.026564,-0.029198,-0.026153,-0.027821,-0.007098,0.019296,-0.099543,-0.029439,0.625934,-0.279258,-0.299801,1.000000,0.445032,0.144252,0.092060,0.622096,0.223562,-0.182457,0.599928,-0.248428,0.586957,0.620977,-0.133424,-0.110222,0.003923,0.602275,-0.026748,-0.270641,0.101853,-0.084902,-0.134409,0.395128,0.999632,0.170788,-0.059079,0.063592,0.053907,0.113160,0.496658,-0.117087,-0.267994
per_capita-Malaria,-0.096207,0.250154,-0.084247,-0.075220,0.113795,0.465031,-0.039844,-0.042953,0.099399,0.226502,-0.074033,0.028765,-0.067598,0.076550,0.076190,-0.070721,-0.052384,-0.009436,0.082190,-0.036464,-0.067351,0.013769,-0.057523,-0.060247,0.006206,0.149687,-0.039595,-0.046935,-0.041164,-0.040628,-0.003961,-0.003490,-0.117623,-0.038859,0.800253,-0.297557,-0.323809,0.445032,1.000000,0.035152,-0.060215,0.685665,0.289723,-0.201828,0.468802,-0.287943,0.651924,0.731067,-0.178402,-0.172920,-0.001716,0.668476,-0.038198,-0.322327,0.080866,-0.158545,-0.202616,0.398795,0.442287,0.150011,-0.210426,0.016136,-0.004486,0.107074,0.393232,-0.110047,-0.225332
per_capita-Drowning,-0.169410,0.004252,-0.018273,-0.005508,0.067014,-0.031274,0.126678,0.050670,0.036457,-0.017398,0.058345,0.050835,0.050279,0.036795,0.023803,0.117416,0.065370,0.042819,0.023453,0.147658,0.010848,-0.024903,-0.009351,-0.008627,0.080706,0.068628,0.040095,0.042522,0.032192,0.028370,0.083987,0.042089,0.207507,0.026064,0.122623,-0.317860,-0.233732,0.144252,0.035152,1.000000,0.181321,0.210646,0.020394,-0.098118,0.182339,0.127444,0.271716,0.171762,0.186572,0.317229,0.052718,0.137574,0.285132,-0.193831,-0.019955,0.221244,0.068139,0.364366,0.141628,0.243414,0.181453,0.199628,0.137977,0.222249,0.182345,-0.114832,-0.383255
per_capita-Interpersonal Violence,-0.021013,-0.018166,-0.062984,-0.065426,-0.003019,-0.028897,-0.026390,0.382283,-0.023247,0.188964,-0.000800,-0.026211,-0.031028,-0.023800,-0.022294,0.117642,-0.019162,0.008983,-0.029122,0.094126,-0.053227,0.009597,-0.004645,-0.024461,-0.002670,0.007922,-0.017800,-0.053329,-0.040449,-0.033288,0.025693,-0.043781,

In [18]:
lst3= []
for i in data_full:
    if "per_capita" not in i and "population" not in i and "gdp/percap"not in i and "country" not in i and len(i)>4:
        lst3.append(i)

In [365]:
lst3

['Meningitis',
 "Alzheimer's Disease and Other Dementias",
 "Parkinson's Disease",
 'Nutritional Deficiencies',
 'Malaria',
 'Drowning',
 'Interpersonal Violence',
 'Maternal Disorders',
 'HIV/AIDS',
 'Drug Use Disorders',
 'Tuberculosis',
 'Cardiovascular Diseases',
 'Lower Respiratory Infections',
 'Neonatal Disorders',
 'Alcohol Use Disorders',
 'Self-harm',
 'Exposure to Forces of Nature',
 'Diarrheal Diseases',
 'Environmental Heat and Cold Exposure',
 'Neoplasms',
 'Conflict and Terrorism',
 'Diabetes Mellitus',
 'Chronic Kidney Disease',
 'Poisonings',
 'Protein-Energy Malnutrition',
 'Road Injuries',
 'Chronic Respiratory Diseases',
 'Cirrhosis and Other Chronic Liver Diseases',
 'Digestive Diseases',
 'Fire, Heat, and Hot Substances',
 'Acute Hepatitis']

In [366]:
data_full.to_csv(f'C:\\Users\\pabli\\Desktop\\Health-Database\data\\ETL\\extraction_data.csv')